In [ ]:
import sys

from PIL import Image
import numpy as np
from einops import rearrange

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.data.image_utils import unpack_float32, equalize_image
from skrgbd.devices.phone import visualize_ir

In [ ]:
camera_scans_dir = '/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/test_calibration/scanning/test_calib_board'
renders_dir = '/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/test_calibration/renders'

In [ ]:
render_scale = 1

# camera = 'tis_left'
# camera = 'tis_right'
# def load_image(i):
#     image = np.asarray(Image.open(f'{camera_scans_dir}/{camera}_i/{i:04}@soft_right@best.png'))
#     image = np.clip(np.mean(image, 2).round(), 0, 255).astype(np.uint8)
#     return image

# camera = f'real_sense_rgb'
# def load_image(i):
#     image = np.asarray(Image.open(f'{camera_scans_dir}/real_sense_i/{i:04}@soft_right@best.png'))
#     image = np.clip(np.mean(image, 2).round(), 0, 255).astype(np.uint8)
#     return image

# camera = f'real_sense_ir'
# def load_image(i):
#     image = np.asarray(Image.open(f'{camera_scans_dir}/real_sense_i/{i:04}@soft_right@best_ir.png'))
#     return image

# camera = f'real_sense_ir_right'
# def load_image(i):
#     image = np.asarray(Image.open(f'{camera_scans_dir}/real_sense_i/{i:04}@soft_right@best_irr.png'))
#     return image

# camera = 'kinect_v2_rgb'
# def load_image(i):
#     image = np.asarray(Image.open(f'{camera_scans_dir}/kinect_v2_i/{i:04}@soft_right.png'))
#     image = np.clip(np.mean(image, 2).round(), 0, 255).astype(np.uint8)
#     return image

# camera = 'kinect_v2_ir'
# def load_image(i):
#     image = unpack_float32(np.asarray(Image.open(f'{camera_scans_dir}/kinect_v2_i/{i:04}_ir.png')))
#     image = equalize_image(image / 65535, 51)
#     image = np.clip((image * 255).round(), 0, 255).astype(np.uint8)
#     return image

# camera = 'phone_left_rgb'
# camera = 'phone_right_rgb'
# def load_image(i):
#     image = np.asarray(Image.open(f'{camera_scans_dir}/{camera[:-4]}_i/{i:04}@soft_right@best.jpg'))
#     image = np.clip(np.mean(image, 2).round(), 0, 255).astype(np.uint8)
#     return image

# camera = 'phone_left_ir'
# camera = 'phone_right_ir'
# render_scale = 4
# def load_image(i):
#     image = np.asarray(Image.open(f'{camera_scans_dir}/{camera[:-3]}_i/{i:04}_ir.png'))
#     image = visualize_ir(image, equalize=True)
#     image = image 
#     return image

In [ ]:
def overlay_images(i):
    image = load_image(i)
    image = np.repeat(image, render_scale, axis=1)
    image = np.repeat(image, render_scale, axis=0)

    render = np.asarray(Image.open(f'{renders_dir}/{camera}/{i:04}.png'))
    render = np.clip(np.mean(render, 2).round(), 0, 255).astype(np.uint8)

    # overlay = ((np.float32(image) - np.float32(render)) + 255) / 2
    # overlay = np.clip(overlay.round(), 0, 255).astype(np.uint8)
    overlay = np.dstack([image, render, image * 0])
    return overlay

In [ ]:
rendered_ids = [0, 11, 12, 23, 24, 35, 36, 47, 48, 58, 59, 69, 70, 80, 81, 90, 91, 99]

In [ ]:
renders = []
for i in rendered_ids:
    try:
        overlay = overlay_images(i)
    except FileNotFoundError:
        overlay = renders[0] * 0 + 255
    renders.append(overlay)

In [ ]:
h, w = renders[0].shape[:2]
window_halfsize = min(h, w) // 20
border = 1

In [ ]:
windows = [render[h // 2 - window_halfsize: h // 2 + window_halfsize,
                  w // 2 - window_halfsize: w // 2 + window_halfsize] for render in renders]

for window in windows:
    window[:border] = 255
    window[-border:] = 255
    window[:, :border] = 255
    window[:, -border:] = 255

In [ ]:
Image.fromarray(rearrange(windows, '(W H) h w c -> (H h) (W w) c', H=2))